In [1]:
from netgen.meshing import Mesh as NGMesh # Vorsicht es gibt Mesh auch in ngsolve!
from netgen.meshing import MeshPoint, Pnt, Element1D, Element0D
from ngsolve import *
from ngsolve.webgui import Draw
import numpy as np
import scipy as sp

def NewtonSteps (a, gfx, nMax=10, tol=1e-9):
    '''
    gfx = Grid function
    The grid function is an approximation of the solution of the boundary value problem.
    This grid function is used for discretising the boundary value problem on the local fine grid.
    '''
    X = gfx.space #Will be defined as a 'raum'
    # vectors for newton steps
    res = gfx.vec.CreateVector()
    du = gfx.vec.CreateVector()

    for it in range (nMax):
        print('Iteration ', it)
        with TaskManager (): #NGSolve supports shared memory parallelization using its built-in TaskManager
            # From here on our code
            #F(u_n) // Chapter 12.1
            a.Apply(gfx.vec, res) #res=Residuum
            
            #F'(u_n) // -> Jacobi-Matrix
            a.AssembleLinearization(gfx.vec) #Bills the second variation -> the jacobi matrix -> linearisation point is the vector 
            #Can be seen in the Gelfand-equation exercise ; Linearisation / derivation of each component will be billed
            inv = a.mat.Inverse(X.FreeDofs())#Feedofs=Freiheitsgrade -> alles ohne Dirchichletrand #inverse() = lineares Gleichungssystem loesen
            
            # F'(u_n) du = F(u_n)
            du.data = inv * res
            
            # update iteration
            # u_n+1 = u_n - du
            gfx.vec.data -= du
            
            #stopping criteria
            stopcritval = sqrt(abs(InnerProduct(du,res)))
            print("Stopcritvalue: " + str(stopcritval))
            if stopcritval < tol:
                break
    if stopcritval < tol:
        return 1 #gfx is global return here means if it worked or not
    else:
        return 0

'''
Trial-Function = Platzhalten für unsere Loesung -> fuer alle TrialFunktionen in Space -> Saemtliche Basisfunktionen sodass Bilinearform ergibt
                 Sobald TrialFunktion in Gleichung drin ist, dann ergibt das eine Matrix eine Linearform -> Dieskretisiert bei Bilinearform immer eine Matrix
Test-Function = Funktionen die man dranmultipliziert -> Saemtliche Basisfunktionen
Bilinear-Form = Schwache Gleichung in unserem Fall -> Gesamte nichtlineare schwache Gleichung, sodass alles gleich 0 ist.
Freiheitsgrade = Unbekannte linear-Faktoren von unseren Grid-Function (Mesh) oder Loesung die wir suchen -> man kann 9 Freiheitsgrade haben aber nur einen Unbekannten wenn die anderen 8 sind durch Rand-bedingung bereits gegeben (bspw. Dirichletrandbedingung)
                 bei Freedofs -> ist es durch den konkatnierten Raum und deren definierten Dirchletraendern so zugeschnitten, damit wir die richtige Anzahl an unbekannten Vars kriegen
ur = Trial-Function of Testfunction vr
vr = Testfunction of Trial-Function ur
uz = Trial-Function of Testfunction vz
vz = Testfunction of Trial-Function uz
p = Trial-Function of Testfunction q
q = Testfunction of Trial-Function p
T = Trial-Function of Testfunction S / or Temperature
S = Testfunction of Trial-Function T
r = Ortsvektor
z = Ortsvektor
X = Concatenated space (W,Vr ,Vz ,Q)
rho = Dichte
alpha = 'warmeausdehnungskoefficient' = 1/T 
kappa = 0.0262W/(mK) = 'Waermeleitfaehigkeit'
g = 9.81m/s^2
gamma = Robin-Randwert -> Wie gross ist der Waermestrom gegen aussen -> im Text ist es betta
Tamb = Umgebungstemperatur
dx = Volumenelement / Integrator
nu = mu / rho ; mu = scaleMu * 18.2e-6 / np.sqrt(293.15) * sqrt(273.15 + T) ; -> dynamische viskositaet / mu=kinetische viskositaet
'''
def stokesFlow(ur, r, vr, T, X, nu, alpha, uz, vz, q, p, rho, g, gamma, Tamb, S, dx, kappa):
    # Stokes flow
    div_u = ur/r+ grad(ur)[0]+ grad(uz)[1]
    div_v = vr/r+ grad(vr)[0]+ grad(vz)[1]

    u = CoefficientFunction ((ur, uz))#For vectorial usage of ur and uz
    conv = u* grad(T)

    a = BilinearForm(X, symmetric = False)
    a += (kappa * grad(T) * grad(S)+ conv * S) * r * dx
    a += gamma *(T- Tamb)*S*r*ds('outer')#Randintegral
    a += (nu*ur/r **2* vr + nu* grad(ur)* grad(vr) + nu * grad(uz ) * grad(vz) + 1/ rho * p * div_v + g * alpha * grad(T)[1] * vz)*r*dx
    a += div_u * q *r * dx
    return a, u, div_u, div_v, conv

def navierStokesFlow(ur, r, vr, T, X, nu, alpha, uz, vz, q, p, rho, g, gamma, Tamb, S, dx, kappa, div_v, div_u):
    #Navier-Stokesflow
    aNS = BilinearForm(X, symmetric=False)
    aNS += (kappa * grad(T) * grad(S) + conv * S) * r * dx
    aNS += gamma * (T - Tamb) * S * r * ds('outer')
    aNS += (nu * ur / r**2 * vr + nu * grad(ur) * grad(vr) + nu * grad(uz) * grad(vz) + u * (grad(ur) * vr + grad(uz) * vz) + 1 / rho * p * div_v + g * alpha * grad(T)[1] * vz) * r * dx
    aNS += div_u * q * r * dx
    return aNS

importing NGSolve-6.2.2204


In [2]:
from netgen . occ import *
from netgen . webgui import Draw as DrawGeo

# Geometry
# wir skalieren die Geometrie um die Rechenbarkeit zu gewaehren
scale = 0.075
geom = MoveTo (0 ,0)
geom = geom.LineTo(.2 ,0)
geom = geom.Spline([(0.85 ,0.7875) ,(1.25 ,1.8) ,(0.9 ,2.5) ,(0 ,2.85)])
geom = geom.Close().Face()
geom.faces.name = 'hotair'
geom.edges.name = 'outer'
geom.edges.Min(X).name = 'sym'
geom.edges.Min(Y).name = 'inlet'
geom = geom.Scale(gp_Pnt((0 ,0 ,0)), scale)
geo = OCCGeometry(geom, dim =2)

# Mesh
mesh = Mesh (geo.GenerateMesh(maxh = scale * 0.025))
mesh.Curve(3)# curved elements on the boundary

In [3]:
#########################################################
####### Finite element Methods - Space definition
#########################################################
order = 2
W = H1(mesh , order=order , dirichlet ="inlet")
Vr = H1(mesh , order=order , dirichlet ="sym|outer|inlet")##orderinner=order+1,//0 auf symettrieachse auf aeusserem Rand und innen dran ist sie auch 0
Vz = H1(mesh , order =order , dirichlet ="outer|inlet")## orderinner=order+1,
Q = H1(mesh , order=order-1) #Lagrange parameter und stroemungfeld matchen -> deswegen 1 ordnung tiefer
X = FESpace ([W,Vr ,Vz ,Q])#W = FE-Space Temperatur, Vr = Radiale Richtung fuer Stroemung, Vz = Z-Richtung fuer Stroemung, Q = Fuer Druck (Fe-Space)
T, ur, uz,p = X.TrialFunction()
S, vr,vz,q = X.TestFunction()
r = x #x = Coeffient function from ngsolve
z = y #y = Coeffient function from ngsolve
#########################################################
#########################################################
#########################################################

In [4]:
T0_d = {"sym" : 0, "outer" : 0, "inlet" : 100}
T0 = CoefficientFunction([T0_d[b] for b in mesh.GetBoundaries()])

In [5]:
#########################################################
####### Material Parameters
#########################################################
kappa = Parameter(0.0262)
scaleMu = Parameter(30)
mu = scaleMu * 18.2e-6 / np.sqrt(293.15) * sqrt(273.15 + T)
rho = 1013.25e-3 * 1e5 / (287.058 * (273.15 + T))
nu = mu / rho
g = 9.81
alpha = 1 / (273.15 + T)
gamma = Parameter(1e-3)
Tamb = Parameter(0)

#### For exercise 4
p0 = 1013.25e-3
#########################################################
#########################################################
#########################################################

In [6]:
### My definitions
gfx = GridFunction(X) #Produktraum -> Konkatenierung der Raeume
############################

#########################################################
####### Grid function
#########################################################
Temp = CoefficientFunction (gfx.components[0])
velocity = CoefficientFunction (gfx.components[1:3])
pressure = CoefficientFunction (gfx.components[3])
gfx.components[0].Set(100,definedon=mesh.Boundaries('inlet'))
#########################################################
#########################################################
#########################################################

In [7]:
#########################################################
####### Implementation of exercise 2.1
#########################################################
(a, u, div_u, div_v, conv) = stokesFlow(ur, r, vr, T, X, nu, alpha, uz, vz, q, p, rho, g, gamma, Tamb, S, dx, kappa)
gfx.components[0].Set(T0,BND)
gfxVecData = NewtonSteps(a, gfx)

Iteration  0
Stopcritvalue: 4.420478670416403
Iteration  1
Stopcritvalue: 0.0041056078838375655
Iteration  2
Stopcritvalue: 1.2445977774604225e-05
Iteration  3
Stopcritvalue: 6.012965523273329e-09
Iteration  4
Stopcritvalue: 1.7849277634557295e-09
Iteration  5
Stopcritvalue: 2.8888032803068165e-09
Iteration  6
Stopcritvalue: 2.2510672586898335e-09
Iteration  7
Stopcritvalue: 1.7392375464506542e-09
Iteration  8
Stopcritvalue: 3.880593926838614e-09
Iteration  9
Stopcritvalue: 2.700109383074952e-09


In [8]:
#########################################################
####### Implementation of exercise 2.1
#########################################################

###### Solution of stokesflow
Draw (Temp, mesh, 'T')
Draw ( velocity, mesh, 'v')

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

BaseWebGuiScene

In [9]:
#########################################################
####### Implementation of exercise 2.1
#########################################################

Draw(gfx.components[0])
velocity = CoefficientFunction(gfx.components[1:3])
Draw(velocity, mesh, "velocity")#Stroemungsfeld

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

BaseWebGuiScene

In [10]:
#########################################################
####### Implementation of exercise 2.2
#########################################################
#oderinnerOorder+1 (Diskretisierung an der liegt das) im H1 -> so kann auch nicht stetig mit L2 gerechnet werden -> fuer die Stroemung 1 order erhoehen dann konvergiert fuer 1
aNS = navierStokesFlow(ur, r, vr, T, X, nu, alpha, uz, vz, q, p, rho, g, gamma, Tamb, S, dx, kappa, div_v, div_u)
gfx.components[0].Set(T0,BND)

# iterativ for Navier - Stokes flow
scaleMus = 1+ np. linspace (0 ,30**(1/2) ,30)[:: -1]**2
print("Das ist die Schrittweise Skalierung: " + str(scaleMus))
for s in scaleMus :
    print(s)
    scaleMu.Set(s)
    ret = NewtonSteps(aNS, gfx, nMax=25, tol=1e-8)
    if ret < 1:
        print('no convergence')
        break

Das ist die Schrittweise Skalierung: [31.         28.9667063  27.00475624 25.11414982 23.29488704 21.5469679
 19.87039239 18.26516052 16.73127229 15.26872771 13.87752675 12.55766944
 11.30915577 10.13198573  9.02615933  7.99167658  7.02853746  6.13674197
  5.31629013  4.56718193  3.88941736  3.28299643  2.74791914  2.28418549
  1.89179548  1.57074911  1.32104637  1.14268728  1.03567182  1.        ]
31.0
Iteration  0
Stopcritvalue: 4.4214976901022105
Iteration  1
Stopcritvalue: 0.0011086238296268691
Iteration  2
Stopcritvalue: 2.5110854599814182e-05
Iteration  3
Stopcritvalue: 1.22516147937401e-07
Iteration  4
Stopcritvalue: 9.852264169072667e-10
28.966706302021404
Iteration  0
Stopcritvalue: 5.4079296983125255e-05
Iteration  1
Stopcritvalue: 4.0050500046707186e-07
Iteration  2
Stopcritvalue: 1.338011796819819e-09
27.00475624256837
Iteration  0
Stopcritvalue: 5.765246154099228e-05
Iteration  1
Stopcritvalue: 5.155738467700775e-07
Iteration  2
Stopcritvalue: 2.139725026651298e-09
25.1141

In [11]:
#########################################################
####### Plot of exercise 2.2
#########################################################
Draw(gfx.components[0])
velocity = CoefficientFunction(gfx.components[1:3])
Draw(velocity, mesh, "velocity")#Stroemungsfeld

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

BaseWebGuiScene

In [12]:
#########################################################
####### Definitions of exercise 3
#########################################################
T, ux, uy,p = X.TrialFunction()
S, vx,vy,q = X.TestFunction()

def stokesFlow2d(ux, vx, T, X, nu, alpha, uy, vy, q, p, rho, g, gamma, Tamb, S, dx, kappa):
    # Stokes flow
    div_u = grad(ux)[0]+ grad(uy)[1]
    div_v = grad(vx)[0]+ grad(vy)[1]

    u = CoefficientFunction ((ux, uy))#For vectorial usage of ur and uz
    conv = u* grad(T)

    a = BilinearForm(X, symmetric = False)
    a += (kappa * grad(T) * grad(S)+ conv * S) * dx
    a += gamma *(T- Tamb)*S*ds('outer')
    a += (nu* grad(ux)* grad(vx) + nu * grad(uy) * grad(vy) + 1/ rho * p * div_v + g * alpha * grad(T)[1] * vy)*dx #bis zu dieser Zeile mit Dozent besprochen
    a += div_u * q * dx
    return a, u, div_u, div_v, conv

def navierStokesFlow2d(ux, vx, T, X, nu, alpha, uy, vy, q, p, rho, g, gamma, Tamb, S, dx, kappa, div_v, div_u):
    #Navier-Stokesflow
    aNS = BilinearForm(X, symmetric=False) #ebensproblem in x, y -> r rausbringen -> z wirds y und r ist das x koordinate
    aNS += (kappa * grad(T) * grad(S) + conv * S) * dx
    aNS += gamma * (T - Tamb) * S * ds('outer')
    aNS += (nu * grad(ux) * grad(vx) + nu * grad(uy) * grad(vy) + u * (grad(ux) * vx + grad(uy) * vy) + 1 / rho * p * div_v + g * alpha * grad(T)[1] * vy) * dx
    aNS += div_u * q * dx
    return aNS
#Rotationssymettrische (axisymetric solution in English) loesung heisst in jedem Winkel phi ist die Loesung genau gleich -> Vorteil nur ein 2D Problem loesen kriegen aber die Loesung fuer ein 3D Problem -> FE kann man das machen aber mit finiten Volumen nicht

In [13]:
#########################################################
####### Implementation of exercise 3 - Stokes-Flow
#########################################################
(a, u, div_u, div_v, conv) = stokesFlow2d(ux, vx, T, X, nu, alpha, uy, vy, q, p, rho, g, gamma, Tamb, S, dx, kappa)
gfu = GridFunction(X)
gfu.components[0].Set(T0,BND)
gfuVecData = NewtonSteps(a, gfu)
velocity = CoefficientFunction(gfx.components[1:3])
Draw(velocity, mesh, "velocity")#Stroemungsfeld

Iteration  0
Stopcritvalue: 49.52082977450688
Iteration  1
Stopcritvalue: 0.24413504797120011
Iteration  2
Stopcritvalue: 0.00264389135846286
Iteration  3
Stopcritvalue: 3.6389119679334467e-06
Iteration  4
Stopcritvalue: 2.3725853418445997e-11


WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

BaseWebGuiScene

In [14]:
#########################################################
####### Implementation exercise 3 - Navier-Stokes-Flow
#########################################################
#oderinnerOorder+1 (Diskretisierung an der liegt das) im H1 -> so kann auch nicht stetig mit L2 gerechnet werden -> fuer die Stroemung 1 order erhoehen dann konvergiert fuer 1
aNS = navierStokesFlow2d(ux, vx, T, X, nu, alpha, uy, vy, q, p, rho, g, gamma, Tamb, S, dx, kappa, div_v, div_u)
gfk = GridFunction(X)
gfk.components[0].Set(T0,BND)

# iterativ for Navier - Stokes flow
scaleMus = 5+ np. linspace (0 ,30**(1/2) ,30)[:: -1]**2
print("Das ist die Schrittweise Skalierung: " + str(scaleMus))
for s in scaleMus :
    print(s)
    scaleMu.Set(s)
    ret = NewtonSteps(aNS, gfk, nMax=25, tol=1e-8)
    if ret < 1:
        print('no convergence')
        break

Das ist die Schrittweise Skalierung: [35.         32.9667063  31.00475624 29.11414982 27.29488704 25.5469679
 23.87039239 22.26516052 20.73127229 19.26872771 17.87752675 16.55766944
 15.30915577 14.13198573 13.02615933 11.99167658 11.02853746 10.13674197
  9.31629013  8.56718193  7.88941736  7.28299643  6.74791914  6.28418549
  5.89179548  5.57074911  5.32104637  5.14268728  5.03567182  5.        ]
35.0
Iteration  0
Stopcritvalue: 49.621616868615064
Iteration  1
Stopcritvalue: 0.019785811537210162
Iteration  2
Stopcritvalue: 3.925351835445179e-05
Iteration  3
Stopcritvalue: 1.7549801657267164e-08
Iteration  4
Stopcritvalue: 2.1044467185675377e-11
32.96670630202141
Iteration  0
Stopcritvalue: 0.00012267426321214015
Iteration  1
Stopcritvalue: 1.1806636704782536e-07
Iteration  2
Stopcritvalue: 2.618350695808991e-11
31.00475624256837
Iteration  0
Stopcritvalue: 0.00012962542287915863
Iteration  1
Stopcritvalue: 1.4517185596317234e-07
Iteration  2
Stopcritvalue: 5.0723954101789735e-12
29.1

In [15]:
#########################################################
####### Plot exercise 3 - Navier-Stokes-Flow
#########################################################
velocity = CoefficientFunction(gfk.components[1:3])
Draw(velocity, mesh, "velocity")#Stroemungsfeld

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

BaseWebGuiScene

In [16]:
#########################################################
####### First bullet point - exercise 4
#########################################################
rhoT = 1013.25e-3*1e5/(287.058*(273.15+Temp)) #Temperaturabhaengige Funktion der Dichte
#10^5 -> heisst bar in pascal umrechnen ; rhoT[kg/m^3]
rho0 = 1013.25e-3*1e5/(287.058*273.15)

massHotAir = 2*np.pi*Integrate(rhoT*r, mesh) #Masse bei warmer Luft
massColdAir = 2*np.pi*Integrate(rho0*r, mesh) #Masse bei kalte Luft

FAuftrieb=(massColdAir-massHotAir)*g
print(g)
print(massHotAir)
print(massColdAir)
print("Die Auftriebskraft lautet: " + str(FAuftrieb) + " N")

9.81
0.003334695866043888
0.004436405617570045
Die Auftriebskraft lautet: 0.010807772662471597 N


In [17]:
#########################################################
####### Second bullet point - exercise 4
#########################################################
#FAuftrieb*hoeheGesucht/hoeheOrig = FG = (200kg+masseHotAir)*g
masse = 200
hOrig = 0.21375 #Original Hoehe -> 2.85(Original-Hoehe * 0.075(Skalierungsfaktor))
alpha = (masse / (massColdAir-massHotAir))**(1/3) #masse die wir heben moechten durch die masse die wir effektiv heben
print("Der Skalierungsfaktor lautet: " + str(alpha))
hNeu = hOrig*alpha
print("Die neue Höhe lautet: " + str(hNeu) + " Meter")

Der Skalierungsfaktor lautet: 56.622312291184095
Die neue Höhe lautet: 12.1030192522406 Meter
